In [13]:
# Importe de Librerías
from google.cloud import storage
import json
import pandas as pd
import gspread


In [3]:
# Definición de variables
json_key = 'top-opus-371913-3159a33cc246.json'
bucket_name = 'pr_hunty'
office_modality_file = 'office_modality.json'
company_size_file = 'tamaño_empresas.json'
email = 'micperezc@gmail.com'
project_name = "hunty_project"

In [4]:
# Conexión a google Storage
storage_client = storage.Client.from_service_account_json(json_key)

In [5]:
# Conexión al Bucket utilizado para cargos los arhivos office_modality.json y tamaño_empresas.json
bucket = storage_client.get_bucket(bucket_name)

In [6]:
# Función para obtener jsons
def get_json(filename):
 
    blob = bucket.get_blob(filename)
    file_data = json.loads(blob.download_as_string())
    return file_data

In [7]:
# Función que crea dataframe de acuerdo a los archivos dispuestos en el bucket: office_modality.json y tamaño_empresas.json
def create_dataframe(file):
     dataset = get_json(file)
     dataset_df = pd.DataFrame.from_records(dataset)
     if 'disponibility' in dataset_df.columns:
        dataset_df['disponibility'] = dataset_df['disponibility'].astype('string')
     return dataset_df

In [8]:
# Creando dataframes con la información de cada una de lis archivos jsons
office_modality_df = create_dataframe(office_modality_file)
company_size_df = create_dataframe(company_size_file)

In [9]:
# Creando el google sheet para alojar la información de los archivos json
service = gspread.service_account(filename=json_key)
hunty_sheet = service.create(project_name)
hunty_sheet.share(email, perm_type='user', role='writer')


<Response [200]>

In [11]:
# Cargando la información de los datos de office modality
hunty_sheet = service.open("hunty_project")
hunty_office_sheet = hunty_sheet.worksheet("Sheet1")
hunty_office_sheet.update([office_modality_df.columns.values.tolist()] + office_modality_df.values.tolist())
hunty_office_sheet.format('A1:E1', {'textFormat': {'bold': True}})

{'spreadsheetId': '1pNi-kflWPmtQXIiIOWTfETsG9yyAnrFCxSCEWaTC9yw',
 'replies': [{}]}

In [12]:
# Cargando la información de los datos de la compañía
hunty_company_sheet = hunty_sheet.add_worksheet(title="Company size", rows="100", cols="20" )
hunty_company_sheet.update([company_size_df.columns.values.tolist()] + company_size_df.values.tolist())
hunty_company_sheet.format('A1:C1', {'textFormat': {'bold': True}})

{'spreadsheetId': '1pNi-kflWPmtQXIiIOWTfETsG9yyAnrFCxSCEWaTC9yw',
 'replies': [{}]}